# Biopharma Case
##### Submission by: Alejandro Ch., Lin Yeh, SJ Ahn, David L., Deep Goon

### Exchange Rate Setup

In [ ]:
!pip install forex_python

from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 1.8 MB/s eta 0:00:00


In [ ]:
def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    # Initializing a currency rates object
    c = CurrencyRates()

    # Key-value pairs of currency and its respective rates
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    # While current date is earlier than the end date, continue the while loop
    while current_date <= end_date:
        for currency in target_currencies: # each currency is its own loop
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                # strftime converts to YYYY-MM -> 2024-02
                rates[currency].append((current_date.strftime("%Y-%m"), rate)) # append a tuple of date and the rate in that specific day
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None)) # if no rate was available on that day, then append the date and None

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

In [ ]:
# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

# Print the rates
for currency, rates in exchange_rates.items():
    print(f"Exchange rates for {currency} against {base_currency}:")
    for date, rate in rates:
        print(f"{date}: {rate}")
    print("\n")

Exchange rates for BRL against USD:
2019-01: 3.8812227074235808
2019-02: 3.6709964257693315
2019-03: 3.7808134938065536
2019-04: 3.8771804912780348
2019-05: 3.9267249064004286
2019-06: 3.9872657160792757
2019-07: 3.8272094457661465
2019-08: 3.8255866630424937
2019-09: 4.157212758245742
2019-10: 4.164433841071756
2019-11: 3.989316814794865
2019-12: 4.230468038608632
2020-01: 4.0196724230016025
2020-02: 4.266829533116178
2020-03: 4.485014120433634
2020-04: 5.2440563277249455
2020-05: 5.384792203015815
2020-06: 5.3324937027707815
2020-07: 5.442857142857142
2020-08: 5.167032410533423
2020-09: 5.433052473512972
2020-10: 5.6008339006126615
2020-11: 5.779363993845102
2020-12: 5.311915106951871
2021-01: 5.193953223046206
2021-02: 5.442320423700762
2021-03: 5.579025968638513
2021-04: 5.631619274646687
2021-05: 5.346548584671412
2021-06: 5.202126789366054
2021-07: 4.960871760350051
2021-08: 5.1060465898578755
2021-09: 5.152407548447152
2021-10: 5.415172413793104
2021-11: 5.650889618241493
2021-1

In [ ]:
data = []
for currency, rates in exchange_rates.items():
    for date, rate in rates:
        data.append({
            'Date': date,
            'Currency': currency,
            'Exchange Rate': rate
        })

# Create DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame to make 'Date' the index and 'Currency' the columns
df_pivoted = df.pivot(index='Date', columns='Currency', values='Exchange Rate')

# Display the pivoted DataFrame
exrates = df_pivoted[df_pivoted.index.str[-2:]=='01']

exrates = exrates.transpose()

exrates.loc["USD"] = [1] * len(exrates.columns)

print(exrates)

Date         2019-01     2020-01     2021-01     2022-01     2023-01
Currency                                                            
BRL         3.881223    4.019672    5.193953    5.571340    5.286518
EUR         0.873362    0.890155    0.814930    0.882924    0.937559
INR        69.633013   71.378850   73.066987   74.368003   82.665479
JPY       109.912664  108.545487  103.080434  115.115663  131.876992
MXN        19.643755   18.889265   19.897319   20.434222   19.553722
USD         1.000000    1.000000    1.000000    1.000000    1.000000


In [ ]:
# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# all() -> returns True if all items in an iterable are true, otherwise it returns False.

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

    print("Mean Vector:\n", mean_vector)
    print("Covariance Matrix:\n", covariance_matrix)
    # The distribution is now defined and can be used for further analysis

Mean Vector:
 [  4.91679403   0.89791993  75.7301565  119.26608249  19.81262121]
Covariance Matrix:
 [[ 3.42594387e-01 -2.13708474e-03  1.22952065e+00  1.27836101e+00
   3.46111090e-01]
 [-2.13708474e-03  2.17002114e-03  1.24913080e-01  5.21137097e-01
  -1.73246912e-02]
 [ 1.22952065e+00  1.24913080e-01  2.02929685e+01  6.06245515e+01
  -2.53203458e+00]
 [ 1.27836101e+00  5.21137097e-01  6.06245515e+01  2.26189966e+02
  -1.36106797e+01]
 [ 3.46111090e-01 -1.73246912e-02 -2.53203458e+00 -1.36106797e+01
   2.22048686e+00]]


### Optimization Model Setup

In [ ]:
!pip install gurobipy
!pip install tabulate

import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum, LinExpr
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 41.3 MB/s eta 0:00:00


In [ ]:
selected_yr = 2023
base_yr = 2019

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18], # Highcal demand
    'd_r': [  7, 12,  3,  8,  3, 17], # Relax demand
})
demand.set_index('from', inplace=True) # sets the index to be the column from

caps = pd.DataFrame({ # capacities
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

# Plant cost matrix -> Table 16.9
pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

# Transportation costs -> Table 16.20
tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

# Import duties -> Table 16.22
duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

# # Commented out to reflect the actual rates
# exrate0 = {
#     '2018': [3.88, 4.33, 69.63, 109.91, 19.64, 1],
#     '2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1],
#     '2020': [5.19, 0.82, 73.66, 103.24, 19.90, 1],
#     '2021': [5.26, 0.88, 74.28, 115.59, 20.62, 1],
#     '2022': [5.29, 0.93, 82.75, 131.12, 19.48, 1],
#     '2023': [4.85, 0.91, 83.04, 140.99, 16.96, 1],
# }
# exrate0 = pd.DataFrame(exrate0 , index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])

#### Cost and Excess Functions

In [ ]:
# identify number of supply and demand location for iterations
n_ctry = range(demand.shape[0]) # number of countries
n_lines = range(demand.shape[1]+1) # three options available, {0, 1},
# 1. plant shut, open
# 2. highcal off, on
# 3. relax off, on

# Objective function to calculate cost
# dec -> represents decision variables
def calc_total_cost(dec_plant, dec_h, dec_r, sampled_ex, base_yr=2019, tariff=0):
    # reshape -> gives a new shape to an array without changing its data, changes shape of dictionary to a matrix
    # Plants Open / Shut, Lines On / Off (B54:D59)
    if isinstance(dec_plant, np.ndarray):
      x_plant = np.array(list(dec_plant.flatten())).reshape(len(n_ctry), len(n_lines))
    else:
      x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    # dec_h -> represents the quantities of highcal shipped from each supplier to each location (G54:L59)
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    # dec_r -> represents the quantities of relax shipped from each supplier to each location (G63:L68)
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    # Base model -> reindx = exrate.loc[:, f'{base_yr}'] / exrate.loc[:, f'{selected_yr}']
    reindx = exrates.loc[:, f'{base_yr}-01'] / sampled_ex

    pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)

    # Matrix of the import duties, plus making the diagonals have no extra import duty hence equal to 1
    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)

    # For the context of the problem there are only tariffs between north america and europe
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    # Variable costs Highcal
    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    # Variable costs Relax
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost




# Calculate excess capacity given decision variables
def calc_excess_cap(dec_plant, dec_h, dec_r):
    condition = False
    if isinstance(dec_plant, np.ndarray):
      x_plant = np.array(list(dec_plant.flatten())).reshape(len(n_ctry), len(n_lines))
      condition = True
    else:
      x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))

    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    excess_cap = (x_plant * caps.values).copy()

    if condition:
      capacities = []
      for i in range(6):
        col = []
        col.append(excess_cap[i][0] - (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))[i])
        col.append(excess_cap[i][1] - np.sum(x_h, axis=1)[i])
        col.append(excess_cap[i][2] - np.sum(x_r, axis=1)[i])
        capacities.append(col)
      return np.array(capacities)
    else:
      excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
      excess_cap[:, 1] -= np.sum(x_h, axis=1)
      excess_cap[:, 2] -= np.sum(x_r, axis=1)
      return excess_cap

# Calculate unmet demand given decision variables
def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = (demand.values).copy()
    unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

    return unmet_demand


#### Gurobi Model

In [ ]:
# # Prompt the user to enter the year
# while True:
#     try:
#         selected_yr = int(input("Enter year (as yyyy, 2023): "))
#         if 2018 <= selected_yr <= 2023:
#             exrate = exrate0.copy()
#             break  # Break the loop if the input is valid
#         else:
#             print("Invalid input. Please enter a year between 2018 and 2023.")
#     except ValueError:
#         print("Invalid input. Please enter a valid year in number (yyyy).")


# while True:
#     try:
#         tariff = float(input("Enter tariff (in percent, e.g. 10 for 10%): "))
#         if 0 <= tariff <= 1000:
#             tariff = tariff/100
#             break  # Break the loop if the input is valid
#         else:
#             print("Invalid input. Please enter a valid number between 0 and 1000.")
#     except ValueError:
#         print("Invalid input. Please enter a valid number.")


# # Create a Gurobi model
# model = Model("MinimizeCost")

# # Assign initial value of decision variables
# dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
# dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
# dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}

# # Define decision variables
# dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
# dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
# dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

# # Excess Capacity constraints
# excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
# for i in n_ctry:
#     for j in n_lines:
#         model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


# # Unmet demand constraints
# unnmet_demand = calc_unmet_demand(dec_h, dec_r)
# for i in n_ctry:
#     for j in range(2):
#         model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")


# # Update the model
# model.update()

# # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
# model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, base_yr, selected_yr, tariff), GRB.MINIMIZE)

# # Suppress optimization output
# model.Params.OutputFlag = 0

# # Optimize the model
# model.optimize()

# # Extract results to print as table
# op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)
# op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
# op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)


# print("\nHighCal Flow\n")
# print(tabulate(op_h, headers='keys', tablefmt='pretty'))
# print("\nRelax Flow\n")
# print(tabulate(op_r, headers='keys', tablefmt='pretty'))
# print("\nStrategy\n")
# print(tabulate(op_plant, headers='keys', tablefmt='pretty'))
# print(f"\nMinimum Cost: $ {round(model.objVal,2)} in year {selected_yr} at Tariff {(tariff*100)}")

### Base Model

In [ ]:
def base_model(ex_rates, base_yr = 2019, tariff=0):
  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
  dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}

  # Define decision variables
  dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")


  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, ex_rates, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  # Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)
  op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  return model.ObjVal, op_plant

### Strategy from Sampling

1. Sample 100 draws from the exchange rate distribution and for each draw determine the optimal network configuration (which plants/lines should be open).

In [ ]:
# Set-up for the simulation table
columns = ["Plant", "H", "R"]
plants = list(caps.index)
features = ["Min_Cost"]

# Setting up the feature names for each speficic plant and the corresponding decision variable
for plant in plants:
  for category in columns:
    features.append(f"{plant}_{category}")

# Number of samples to be drawn
n_samples = 100

# List of lists, where each list is the output of a simulation
simulations = []

# Simulate by the number of samples to be drawn
for sim in range(n_samples):
  results = [] # initialize empty list to hold the results of single iteration of the base model
  sample = mvn_distribution.rvs(size=1) # sample a set of exchange rates
  sample_matching = np.append(sample, 1) # append a 1 to match function definition within calc_total_cost fucntion
  model_output = base_model(sample_matching) # pass the sampled exchange rate as a parameter to the model

  # Flatten matrix op_plant which contains the decision variables on a plant-level
  results = model_output[1].values.flatten().tolist()
  # Insert the minimum cost calculated by the network
  results.insert(0, round(model_output[0],2))
  # Append the results to simulation
  simulations.append(results)

# Convert the lists of lists into a matrix object in numpy
data_array = np.array(simulations)
# Fill in a new DataFrame where each row represents a simulation and how the plants behaved
df_simulation = pd.DataFrame(data_array, columns=features)

frequencies = df_simulation.mean()

frequencies



Restricted license - for non-production use only - expires 2025-11-24


Min_Cost         1187.9038
Brazil_Plant        1.0000
Brazil_H            1.0000
Brazil_R            1.0000
Germany_Plant       1.0000
Germany_H           0.0700
Germany_R           0.9800
India_Plant         1.0000
India_H             1.0000
India_R             1.0000
Japan_Plant         0.5500
Japan_H             0.3600
Japan_R             0.4300
Mexico_Plant        0.7700
Mexico_H            0.7700
Mexico_R            0.7700
U.S._Plant          0.8100
U.S._H              0.7900
U.S._R              0.1000
dtype: float64

#### Candidate Proposal
Based on the results from #1, propose 3 candidate network strategies we can expect to work well. Use the following logic: if a plant/line is open in most of the scenarios, it is reasonable to keep it open in your proposed strategy, if it is mostly closed, then it makes sense to keep it closed. Consider whether some plants open/close in pairs due to the correlation between exchange rates: for example, if Germany is open, then Japan may be closed. Can you identify such pairs?

## **Strategy 1** - (Frequency Strategy): Keep plants that have a higher frequency of being open. In essence, only plants that are open 60% of the time remain open in the optimal network.

In [73]:
frequencies

Min_Cost         1187.9038
Brazil_Plant        1.0000
Brazil_H            1.0000
Brazil_R            1.0000
Germany_Plant       1.0000
Germany_H           0.0700
Germany_R           0.9800
India_Plant         1.0000
India_H             1.0000
India_R             1.0000
Japan_Plant         0.5500
Japan_H             0.3600
Japan_R             0.4300
Mexico_Plant        0.7700
Mexico_H            0.7700
Mexico_R            0.7700
U.S._Plant          0.8100
U.S._H              0.7900
U.S._R              0.1000
dtype: float64

With a minimum threshold of 60%, here are the following frequency decisions:


*   **Open**: Brazil (both), Germany (only Relax), India (both), Mexico (both), U.S (only Highcal)
*   **Close**: Japan



In [ ]:
def frequency_model(ex_rates, base_yr = 2019, tariff=0):
  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = np.array([[1, 1, 1], # Brazil
                      [1, 0, 1], # Germany
                      [1, 1, 1], # India
                      [0, 0, 0], # Japan
                      [1, 1, 1], # Mexico
                      [1, 1, 0]], dtype='int64') # U.S.

  dec_h     = {(i, j): 0 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 0 for i in n_ctry for j in n_ctry}


  # Define decision variables
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, ex_rates, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  if model.Status == GRB.OPTIMAL:
    # Proceed to access .x attributes
    pass
  else:
      print("Model not solved optimally. Status:", model.Status)

  # Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j] for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index) # Removed .x as dec_plant no longer contains decision variables
  op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  return model.ObjVal, op_plant


2. Correlation between Plants: The following approach tries to identify whether a plant opens or closes based on the status of another plant.

In [ ]:
df_simulation

,Min_Cost,Brazil_Plant,Brazil_H,Brazil_R,Germany_Plant,Germany_H,Germany_R,India_Plant,India_H,India_R,Japan_Plant,Japan_H,Japan_R,Mexico_Plant,Mexico_H,Mexico_R,U.S._Plant,U.S._H,U.S._R
0,1118.38,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
1,1193.52,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1246.43,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,-0.0
3,1185.60,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
4,1217.81,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1147.30,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
96,1231.06,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
97,1219.83,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
98,1114.92,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
df_simulation.iloc[:, 1:19].corr()

# df_simulation.mean()

,Brazil_Plant,Brazil_H,Brazil_R,Germany_Plant,Germany_H,Germany_R,India_Plant,India_H,India_R,Japan_Plant,Japan_H,Japan_R,Mexico_Plant,Mexico_H,Mexico_R,U.S._Plant,U.S._H,U.S._R
Brazil_Plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brazil_H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brazil_R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Germany_Plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Germany_H,NaN,NaN,NaN,NaN,1.000000,-0.520708,NaN,NaN,NaN,0.090598,-0.205764,0.157540,-0.315718,-0.315718,-0.315718,0.132875,-0.050999,0.169837
Germany_R,NaN,NaN,NaN,NaN,-0.520708,1.000000,NaN,NaN,NaN,0.157935,0.107143,0.124079,-0.078077,-0.078077,-0.078077,-0.069189,0.277080,-0.428571
India_Plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
India_H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
India_R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Japan_Plant,NaN,NaN,NaN,NaN,0.090598,0.157935,NaN,NaN,NaN,1.000000,0.678401,0.785637,-0.494360,-0.494360,-0.494360,-0.386848,-0.318309,-0.167506


The previous output demonstrates the correlation between plants being open for the 100 simulation of the base model.


*  **Japan and Mexico**: The negative moderate correlation between Japan_Plant and Mexico_Plant demonstrates that whenever one of these plants is open, the other high has a higher chance of being closed.
*   **Japan and U.S.**: The negative weak correlation between Japan_Plant and U.S._Plant demonstrates that whenever one of these plants is open, the other high has a higher chance of being closed.



In [ ]:
def plantcorr_model(ex_rates, base_yr = 2019, tariff=0):
  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = np.array([[1, 1, 1], # Brazil
                      [1, 0, 1], # Germany
                      [1, 1, 1], # India
                      [0, 0, 0], # Japan
                      [1, 1, 1], # Mexico
                      [1, 1, 0]], dtype='int64') # U.S.

  dec_h     = {(i, j): 0 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 0 for i in n_ctry for j in n_ctry}


  # Define decision variables
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, ex_rates, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  if model.Status == GRB.OPTIMAL:
    # Proceed to access .x attributes
    pass
  else:
      print("Model not solved optimally. Status:", model.Status)

  # Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j] for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index) # Removed .x as dec_plant no longer contains decision variables
  op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  return model.ObjVal, op_plant



3. Correlation between Exhange Rates: The following approach tries to identify whether a plant opens or closes in a region based on the correlation of exchange rates.

In [ ]:
exrates.T.corr()

Currency,BRL,EUR,INR,JPY,MXN,USD
Currency,,,,,,
BRL,1.000000,0.020341,0.647313,0.384891,0.717071,NaN
EUR,0.020341,1.000000,0.636011,0.885894,-0.274118,NaN
INR,0.647313,0.636011,1.000000,0.890058,0.106674,NaN
JPY,0.384891,0.885894,0.890058,1.000000,0.027394,NaN
MXN,0.717071,-0.274118,0.106674,0.027394,1.000000,NaN
USD,NaN,NaN,NaN,NaN,NaN,NaN


### Performance Evaluation

Evaluate performance of the three proposed strategies by drawing 100 exchange rate samples and computing expected strategy cost/and cost standard deviation over these samples. Note that the network should remain fixed in this analysis (i.e., plant/lines decision variables must be excluded from the optimization -- you need to modify the Biopharma notebook accordingly). Which strategy do you ultimately recommend?

In [ ]:
def sim(n_samples, strategy):
  # List of lists, where each list is the output of a simulation
  simulations = []

  # Simulate by the number of samples to be drawn
  for sim in range(n_samples):
    results = [] # initialize empty list to hold the results of single iteration of the base model
    sample = mvn_distribution.rvs(size=1) # sample a set of exchange rates
    sample_matching = np.append(sample, 1) # append a 1 to match function definition within calc_total_cost fucntion
    model_output = strategy(sample_matching) # pass the sampled exchange rate as a parameter to the model

    # Flatten matrix op_plant which contains the decision variables on a plant-level
    results = model_output[1].values.flatten().tolist()
    # Insert the minimum cost calculated by the network
    results.insert(0, round(model_output[0],2))
    # Append the results to simulation
    simulations.append(results)

  # Convert the lists of lists into a matrix object in numpy
  data_array = np.array(simulations)
  # Fill in a new DataFrame where each row represents a simulation and how the plants behaved
  df_simulation = pd.DataFrame(data_array, columns=features)

  return df_simulation.round(3)


In [ ]:
print("======================= Frequency Approach =======================")
freq_strategy = sim(100, frequency_model)
print(f'Expected Strategy Cost = {round(freq_strategy["Min_Cost"].mean(), 3)}\nCost Standard Deviation = {round(freq_strategy["Min_Cost"].std(), 3)} ')


======================= Frequency Approach =======================
Expected Strategy Cost = 1196.174
Cost Standard Deviation = 43.217 


In [ ]:
frequencies

Min_Cost         1187.9038
Brazil_Plant        1.0000
Brazil_H            1.0000
Brazil_R            1.0000
Germany_Plant       1.0000
Germany_H           0.0700
Germany_R           0.9800
India_Plant         1.0000
India_H             1.0000
India_R             1.0000
Japan_Plant         0.5500
Japan_H             0.3600
Japan_R             0.4300
Mexico_Plant        0.7700
Mexico_H            0.7700
Mexico_R            0.7700
U.S._Plant          0.8100
U.S._H              0.7900
U.S._R              0.1000
dtype: float64

## **Strategy 2**: Keep Japan and US plants closed

Based on our previous frequency based strategy, we can see on average US is open the least after Japan.
We can implement the following strategy:

*   **Open**: Brazil (both), Germany (only Relax), India (both), Mexico (both)
* **Closed**: US, Japan



In [ ]:
def frequency_model_2(ex_rates, base_yr = 2019, tariff=0):
  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = np.array([[1, 1, 1], # Brazil
                      [1, 0, 1], # Germany
                      [1, 1, 1], # India
                      [0, 0, 0], # Japan
                      [1, 1, 1], # Mexico
                      [0, 0, 0]], dtype='int64') # U.S.

  dec_h     = {(i, j): 0 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 0 for i in n_ctry for j in n_ctry}


  # Define decision variables
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, ex_rates, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  if model.Status == GRB.OPTIMAL:
    # Proceed to access .x attributes
    pass
  else:
      print("Model not solved optimally. Status:", model.Status)

  # Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j] for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index) # Removed .x as dec_plant no longer contains decision variables
  op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  return model.ObjVal, op_plant


In [ ]:
print("======================= Frequency Approach =======================")
freq_strategy = sim(100, frequency_model_2)
print(f'Expected Strategy Cost = {round(freq_strategy["Min_Cost"].mean(), 3)}\nCost Standard Deviation = {round(freq_strategy["Min_Cost"].std(), 3)} ')

======================= Frequency Approach =======================
Expected Strategy Cost = 1213.178
Cost Standard Deviation = 43.412 


## **Strategy 3**: Keep Japan and India plants closed




In [ ]:
covariance_matrix.diagonal()

array([3.42594387e-01, 2.17002114e-03, 2.02929685e+01, 2.26189966e+02,
       2.22048686e+00])

Based on the exchange rate covariance matrix, we can see that the 4th and 3rd currencies have the highest volatility (highest standard deviation). The 4th and 3rd currencies in the diagonal of the covariance matrix below correspond to Japan and India respectively. Thus, we can try a strategy of keeping these plants close to avoid dealing with their volatile currency.

In [ ]:
def frequency_model_3(ex_rates, base_yr = 2019, tariff=0):
  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = np.array([[1, 1, 1], # Brazil
                      [1, 0, 1], # Germany
                      [0, 0, 0], # India
                      [0, 0, 0], # Japan
                      [1, 1, 1], # Mexico
                      [1, 1, 0]], dtype='int64') # U.S.

  dec_h     = {(i, j): 0 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 0 for i in n_ctry for j in n_ctry}


  # Define decision variables
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, ex_rates, base_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

  if model.Status == GRB.OPTIMAL:
    # Proceed to access .x attributes
    pass
  else:
      print("Model not solved optimally. Status:", model.Status)

  # Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j] for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index) # Removed .x as dec_plant no longer contains decision variables
  op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
  op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

  return model.ObjVal, op_plant


In [ ]:
print("======================= Frequency Approach =======================")
freq_strategy = sim(100, frequency_model_3)
print(f'Expected Strategy Cost = {round(freq_strategy["Min_Cost"].mean(), 3)}\nCost Standard Deviation = {round(freq_strategy["Min_Cost"].std(), 3)} ')

======================= Frequency Approach =======================
Expected Strategy Cost = 1244.047
Cost Standard Deviation = 46.824 


## **Recommendation**: Based on the simulations and numbers above, we would ultimately recommend **Strategy 1** (Closing Japan and keeping other plants open) as it yields the lowest expected strategy cost and has a relatively low standard deviation (more stable)